### Produce Airline standings

In [0]:
%run "../includes/configurations"

In [0]:
from pyspark.sql.functions import col, current_timestamp, sum, count, desc, rank
from pyspark.sql.window import Window

In [0]:
flight_search_df = spark.read.parquet(f"{presentation_folder_path}search_flight")

In [0]:
display(flight_search_df)

In [0]:
airline_standings = (flight_search_df.
                     groupBy("year","airline_name").
                     agg(
                         sum("distance").alias("total_distance"),
                         count("*").alias("total_flights")
                     )
                    )

In [0]:
airline_rank_desc = Window.partitionBy("year").orderBy(desc("total_flights"), desc("total_distance"))
final_df = airline_standings.withColumn("rank", rank().over(airline_rank_desc))

In [0]:
# final_df.write.mode("overwrite").partitionBy("year").parquet(f"{presentation_folder_path}airline_standings")

In [0]:
(final_df.
 write.mode("overwrite").
 partitionBy("year").
 format("parquet").
 saveAsTable("dev_air_travel_present.airline_standings")
)